# Neo4j Graph Sampling for PyG GNN 
__An example using the [`ogbn-arxiv`](https://ogb.stanford.edu/docs/nodeprop/#ogbn-arxiv) dataset__

![Neo4j version](https://img.shields.io/badge/Neo4j-5-brightgreen)
![GDS version](https://img.shields.io/badge/GDS-2.3-brightgreen)
![GDS Python Client version](https://img.shields.io/badge/GDS_Python_Client-1.6-brightgreen)

This notebook demonstrates how to use [Neo4j Graph Data Science](https://neo4j.com/docs/graph-data-science/current/) and PyTorch Geometric (PyG) to:

- Generate a representative sample of a graph database using Graph Data Science [projections](https://neo4j.com/docs/graph-data-science/current/management-ops/projections/graph-project/) and [random walk with restarts sampling algorithm](https://neo4j.com/docs/graph-data-science/current/management-ops/projections/rwr/)
- Export the graph sample to Python dataframes
- Define and train a Graph Convolutional Neural Network (GCN) on the sample
- Evaluate the GCN on a test set

## Prerequisites:
To generate the Neo4j Graph Database from `ogbn-arxiv`, please first run `db-load.ipynb`.

## Setup

In [1]:
%pip install graphdatascience python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [3]:
load_dotenv('db-credentials.env', override=True)

# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")
PROJ_NAME = 'proj'

In [4]:
gds.version()

'2.3.1'

In [5]:
RANDOM_SEED = 7474
VALID_YEAR = 2018
SAMPLE_RATIO = 0.34

## Sample Neo4j Graph

In [6]:
if gds.graph.exists(PROJ_NAME)['exists']:
    gds.graph.get(PROJ_NAME).drop()

In [7]:
%%time
g, _ = gds.graph.project(PROJ_NAME, ['Train', 'Valid', 'Test'], ['CITES'], 
                         nodeProperties =['textEmbedding', 'subjectId', 'year'])

Loading:   0%|          | 0/100 [00:00<?, ?%/s]

CPU times: user 27.9 ms, sys: 0 ns, total: 27.9 ms
Wall time: 1.32 s


In [8]:
print(f"Number of nodes in our graph: {g.node_count():,}")
print(f"Number of relationships in our graph: {g.relationship_count():,}")

Number of nodes in our graph: 169,343
Number of relationships in our graph: 1,166,243


In [9]:
SAMPLE_PROJ_NAME = PROJ_NAME + '_sample'
if gds.graph.exists(SAMPLE_PROJ_NAME)['exists']:
    gds.graph.get(SAMPLE_PROJ_NAME).drop()

In [10]:
%%time
g_sample, _ = gds.alpha.graph.sample.rwr(SAMPLE_PROJ_NAME, g, samplingRatio=SAMPLE_RATIO,
                                         restartProbability=0.05, nodeLabelStratification=True, 
                                         concurrency=1, randomSeed=RANDOM_SEED)

Random walk with restarts sampling:   0%|          | 0/100 [00:00<?, ?%/s]

CPU times: user 448 ms, sys: 32.7 ms, total: 480 ms
Wall time: 1min


In [11]:
# If our SAMPLE_RATIO is 0.34 we should have somewhere around 0.34 * 269,343 ~ 55,883 nodes in our sample
print(f"Number of nodes in our sample: {g_sample.node_count():,}")

# And let's see how many relationships we have
print(f"Number of relationships in our sample: {g_sample.relationship_count():,}")

Number of nodes in our sample: 57,577
Number of relationships in our sample: 459,928


In [12]:
# We should also see similar degree distributions
print(f"Degree distribution in our full graph:")
print(g.degree_distribution())

# And let's see how many relationships we got
print(f"\n\nDegree distribution in our sample:")
print(g_sample.degree_distribution())

Degree distribution in our full graph:
p99      37.000000
min       0.000000
max     436.000000
mean      6.886869
p90      18.000000
p50       4.000000
p999     71.000000
p95      24.000000
p75       9.000000
dtype: float64


Degree distribution in our sample:
p99      40.000000
min       0.000000
max     403.000000
mean      7.988051
p90      20.000000
p50       5.000000
p999     82.000000
p95      26.000000
p75      11.000000
dtype: float64


## Export Sampled Graph to Pandas DataFrames

In [13]:
raw_sample_topology_df = gds.beta.graph.relationships.stream(g_sample)

In [14]:
raw_sample_node_df = gds.graph.nodeProperties.stream(
    g_sample,
    ['subjectId', 'year', 'textEmbedding'],
    separate_property_columns=True,
)

## Re-Index Graph Data for PyG
Since we sampled from our Neo4j Graph, we have some missing node ids. This will create issues when trying to transform our data for PyG. To resolve, we will re-assign sequential node ids for the node and relationship topology dataframes.

In [15]:
sample_node_df = raw_sample_node_df.reset_index().rename(columns={'nodeId':'neo4jNodeId'}).rename(columns={'index':'nodeId'})
sample_node_df

,nodeId,neo4jNodeId,subjectId,year,textEmbedding
0,0,112370,10,2012,"[-0.03758800029754639, -0.019776999950408936, ..."
1,1,112371,10,2013,"[-0.09249299764633179, 0.06776200234889984, -0..."
2,2,112375,10,2011,"[-0.05762699991464615, -0.02996399998664856, -..."
3,3,112376,10,2013,"[-0.010092999786138535, -0.06655500084161758, ..."
4,4,112380,10,2000,"[-0.0767190009355545, -0.13565599918365479, -0..."
...,...,...,...,...,...
57572,57572,112360,10,2013,"[-0.25040701031684875, -0.15918700397014618, -..."
57573,57573,112362,10,2013,"[-0.31519201397895813, -0.06957100331783295, -..."
57574,57574,112363,10,2014,"[-0.32545000314712524, -0.3242749869823456, -0..."
57575,57575,112364,10,2012,"[-0.19740299880504608, -0.05031999945640564, -..."


In [16]:
sample_topology_df = (raw_sample_topology_df
    .merge(sample_node_df[['neo4jNodeId','nodeId']], how='left', 
           left_on='sourceNodeId', right_on='neo4jNodeId')
    .drop(columns=['sourceNodeId', 'neo4jNodeId'])
    .rename(columns={'nodeId':'sourceNodeId'})
    .merge(sample_node_df[['neo4jNodeId','nodeId']], how='left', 
           left_on='targetNodeId', right_on='neo4jNodeId')
    .drop(columns=['targetNodeId', 'neo4jNodeId'])
    .rename(columns={'nodeId':'targetNodeId'})
)
sample_topology_df

,relationshipType,sourceNodeId,targetNodeId
0,CITES,0,35957
1,CITES,1,57217
2,CITES,2,30460
3,CITES,2,30461
4,CITES,3,57536
...,...,...,...
459923,CITES,37575,50806
459924,CITES,37576,12136
459925,CITES,37576,16918
459926,CITES,37576,16977


## Construct Inputs for Training
Now that we re-assigned node ids, we can easily transform our `sample_topology_df` and `node_df` into the `edge_index`, features (`x`), and target (`y`) tensors for PyG. We will also use `node_df.year` for data splitting.

In [17]:
# By using `by_rel_type` we get the topology in a format that can be used as input to several GNN frameworks:
# {"rel_type": [[source_nodes], [target_nodes]]}
sample_topology = sample_topology_df.by_rel_type()

In [18]:
edge_index = torch.tensor(sample_topology['CITES'], dtype=torch.long)
edge_index

tensor([[    0,     1,     2,  ..., 37576, 37576, 37576],
        [35957, 57217, 30460,  ..., 16918, 16977,  6490]])

In [19]:
#node features
x = torch.tensor(np.stack(sample_node_df['textEmbedding']), dtype=torch.float)

In [20]:
#target class
y = torch.tensor(np.stack(sample_node_df['subjectId']), dtype=torch.long)

In [21]:
# data objects and masks for data splitting
data = Data(x=x, y=y, edge_index=edge_index)
data.train_mask = torch.tensor(np.stack(sample_node_df.year < VALID_YEAR))
data.val_mask = torch.tensor(np.stack(sample_node_df.year == VALID_YEAR))
data.test_mask = torch.tensor(np.stack(sample_node_df.year > VALID_YEAR))
print(data)

Data(x=[57577, 128], edge_index=[2, 459928], y=[57577], train_mask=[57577], val_mask=[57577], test_mask=[57577])


In [22]:
num_classes = y.unique().shape[0]
print(f'there are {num_classes} possible target classes')

there are 40 possible target classes


## Define Convolutional Neural Network and Other Configurations for Training

In [23]:
# Define the GCN architecture
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 72)
        self.conv2 = GCNConv(72, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # We use log_softmax and nll_loss instead of softmax output and cross entropy loss
        # for reasons for performance and numerical stability.
        # They are mathematically equivalent
        return F.log_softmax(x, dim=1)

In [24]:
# Prepare training by setting up for the chosen device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Let's see what device was chosen
print(device)

cuda


In [25]:
# In standard PyTorch fashion we instantiate our model, and transfer it to the memory of the chosen device
model = GCN().to(device)

# Let's inspect our model architecture
print(model)

GCN(
  (conv1): GCNConv(128, 72)
  (conv2): GCNConv(72, 40)
)


In [26]:
# Pass our input data to the chosen device too
data = data.to(device)

In [27]:
# Since hyperparameter tuning is out of scope for this small example, we initialize an
# Adam optimizer with some fixed learning rate and weight decay
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

## Train and Evaluate GNN

In [28]:
# Train the GCN using the CORA sample represented by `data` using the standard PyTorch training loop
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    train_loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    valid_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask])
    test_loss = F.nll_loss(out[data.test_mask], data.y[data.test_mask])
    print(f'Epoch: {epoch:03d}, '
      f'Train: {train_loss:.4f}, '
      f'Valid: {valid_loss:.4f}, '
      f'Test: {test_loss:.4f}')
    valid_loss.backward()
    optimizer.step()

Epoch: 000, Train: 3.8403, Valid: 3.8400, Test: 3.7851
Epoch: 001, Train: 3.4980, Valid: 3.3984, Test: 3.4800
Epoch: 002, Train: 3.3696, Valid: 3.0955, Test: 3.2269
Epoch: 003, Train: 3.3504, Valid: 2.8919, Test: 2.9968
Epoch: 004, Train: 3.3218, Valid: 2.7671, Test: 2.8403
Epoch: 005, Train: 3.2921, Valid: 2.6941, Test: 2.7497
Epoch: 006, Train: 3.2470, Valid: 2.6350, Test: 2.7085
Epoch: 007, Train: 3.2105, Valid: 2.6195, Test: 2.6972
Epoch: 008, Train: 3.1645, Valid: 2.5798, Test: 2.6555
Epoch: 009, Train: 3.1240, Valid: 2.5270, Test: 2.6254
Epoch: 010, Train: 3.0777, Valid: 2.4804, Test: 2.5802
Epoch: 011, Train: 3.0208, Valid: 2.4354, Test: 2.5477
Epoch: 012, Train: 2.9668, Valid: 2.4006, Test: 2.5322
Epoch: 013, Train: 2.9109, Valid: 2.3584, Test: 2.5121
Epoch: 014, Train: 2.8518, Valid: 2.3077, Test: 2.4973
Epoch: 015, Train: 2.8146, Valid: 2.2906, Test: 2.4759
Epoch: 016, Train: 2.7751, Valid: 2.2649, Test: 2.4597
Epoch: 017, Train: 2.7239, Valid: 2.2338, Test: 2.4289
Epoch: 018

In [29]:
# Evaluate the trained GCN model on our test set
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())

print(f"Test Set Accuracy: {acc:.4f}")   

Test Set Accuracy: 0.5768


## Attempt with Full Graph
Let's try again with the full graph as a quick sanity check.  The final accuracy should be close

In [ ]:
raw_topology_df = gds.beta.graph.relationships.stream(g)

raw_node_df = gds.graph.nodeProperties.stream(
    g,
    ['subjectId', 'year', 'textEmbedding'],
    separate_property_columns=True,
)

node_df = raw_node_df.reset_index().rename(columns={'nodeId':'neo4jNodeId'}).rename(columns={'index':'nodeId'})

topology_df = (raw_topology_df
    .merge(node_df[['neo4jNodeId','nodeId']], how='left', left_on='sourceNodeId', 
           right_on='neo4jNodeId')
    .drop(columns=['sourceNodeId', 'neo4jNodeId'])
    .rename(columns={'nodeId':'sourceNodeId'})
    .merge(node_df[['neo4jNodeId','nodeId']], how='left', left_on='targetNodeId', 
           right_on='neo4jNodeId')
    .drop(columns=['targetNodeId', 'neo4jNodeId'])
    .rename(columns={'nodeId':'targetNodeId'})
)

topology = topology_df.by_rel_type()

edge_index = torch.tensor(topology['CITES'], dtype=torch.long)

x = torch.tensor(np.stack(node_df['textEmbedding']), dtype=torch.float)
y = torch.tensor(np.stack(node_df['subjectId']), dtype=torch.long)

full_data = Data(x=x, y=y, edge_index=edge_index)
full_data.train_mask = torch.tensor(np.stack(node_df.year < VALID_YEAR))
full_data.val_mask = torch.tensor(np.stack(node_df.year == VALID_YEAR))
full_data.test_mask = torch.tensor(np.stack(node_df.year > VALID_YEAR))
print(full_data)

num_classes = y.unique().shape[0]
print(f'there are {num_classes} possible target classes')

full_model = GCN().to(device)
print(full_model)

full_data = full_data.to(device)

optimizer = torch.optim.Adam(full_model.parameters(), lr=0.01, weight_decay=5e-4)

full_model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = full_model(full_data)
    train_loss = F.nll_loss(out[full_data.train_mask], full_data.y[full_data.train_mask])
    valid_loss = F.nll_loss(out[full_data.val_mask], full_data.y[full_data.val_mask])
    test_loss = F.nll_loss(out[full_data.test_mask], full_data.y[full_data.test_mask])
    valid_loss.backward()
    optimizer.step()

In [ ]:
# Evaluate the full data trained GCN model on our test set
full_model.eval()
full_pred = full_model(full_data).argmax(dim=1)
full_correct = (full_pred[full_data.test_mask] == full_data.y[full_data.test_mask]).sum()
full_acc = int(full_correct) / int(full_data.test_mask.sum())

print(f'Test set accuracy for full dataset: {full_acc:.4f}')
print(f'This is a difference of {full_acc-acc:.4f} percentage points from the sampled dataset')

## Cleanup
Remove Neo4j graph projections to free up memory

In [ ]:
g_sample.drop()
g.drop()